In [1]:
import pandas as pd
import utils

# Loading, cleaning, and merging the data

In [2]:
pbp = utils.get_data("pbp")
weather = utils.get_data("weather")
pbp = utils.clean_data("pbp", pbp)
weather = utils.clean_data("weather", weather)
FullData = pbp.merge(weather, how = 'inner', on = ('game_date', 'home_team'))

C:\Users\Michael\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Michael\Desktop\602\Project 1\Notebooks\utils.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data[["posteam", "game_date", "half_seconds_remaining", "game_half", "field_goal_result", "kick_distance", "score_differential", "kicker_player_name", "kicker_player_id", "home_team"]].loc[data.play_type == "field_goal"][data.game_date > "2014-08-01"]


In [3]:
FullData

,posteam,game_date,half_seconds_remaining,game_half,field_goal_result,kick_distance,score_differential,kicker_player_name,kicker_player_id,home_team,stadium,weather_temperature,weather_wind_mph,weather_detail
0,SEA,2014-09-04,1267.0,Half1,made,35.0,0.0,S.Hauschka,00-0025944,SEA,CenturyLink Field,70.0,5.0,Normal
1,GB,2014-09-04,415.0,Half1,made,23.0,-3.0,M.Crosby,00-0025580,SEA,CenturyLink Field,70.0,5.0,Normal
2,SEA,2014-09-04,1532.0,Half2,made,20.0,7.0,S.Hauschka,00-0025944,SEA,CenturyLink Field,70.0,5.0,Normal
3,NO,2014-09-07,1538.0,Half1,made,31.0,0.0,S.Graham,00-0019536,ATL,Georgia Dome,72.0,0.0,DOME
4,NO,2014-09-07,1184.0,Half1,made,50.0,3.0,S.Graham,00-0019536,ATL,Georgia Dome,72.0,0.0,DOME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4876,PHI,2018-12-16,1097.0,Half2,made,40.0,7.0,J.Elliott,00-0033787,LA,Los Angeles Memorial Coliseum,60.0,5.0,Normal
4877,LA,2018-12-16,568.0,Half2,made,37.0,-17.0,G.Zuerlein,00-0029621,LA,Los Angeles Memorial Coliseum,60.0,5.0,Normal
4878,PHI,2018-12-16,73.0,Half2,missed,53.0,7.0,J.Elliott,00-0033787,LA,Los Angeles Memorial Coliseum,60.0,5.0,Normal
4879,NO,2018-12-17,1040.0,Half1,made,46.0,-7.0,W.Lutz,00-0032569,CAR,Bank of America Stadium,48.0,3.0,Normal


# Finding the amounts of made and missed field goals, and calculating the percentage of made field goals in the data

In [4]:
FullData.loc[:, "field_goal_result"].value_counts()

made      4122
missed     759
Name: field_goal_result, dtype: int64

In [5]:
4122/(759+4122)

0.844499078057775

# Checking for null values

In [6]:
FullData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4881 entries, 0 to 4880
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   posteam                 4881 non-null   object        
 1   game_date               4881 non-null   datetime64[ns]
 2   half_seconds_remaining  4881 non-null   float64       
 3   game_half               4881 non-null   object        
 4   field_goal_result       4881 non-null   object        
 5   kick_distance           4881 non-null   float64       
 6   score_differential      4881 non-null   float64       
 7   kicker_player_name      4881 non-null   object        
 8   kicker_player_id        4881 non-null   object        
 9   home_team               4881 non-null   object        
 10  stadium                 4881 non-null   object        
 11  weather_temperature     4881 non-null   float64       
 12  weather_wind_mph        4881 non-null   float64 

# Seperating the predictors and the target and using LabelEncoder to encode the target

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
LabEnc = LabelEncoder()
LabEnc.fit(FullData.loc[:, "field_goal_result"])
y = LabEnc.transform(FullData.loc[:, "field_goal_result"])
X = FullData[["posteam", "half_seconds_remaining", "game_half", "kick_distance", "score_differential", "kicker_player_name", "stadium", "weather_temperature", "weather_wind_mph", "weather_detail"]]

In [9]:
#After encoding, a result of made is given the value of 0, while a result of missed is given the value of 1
y

array([0, 0, 0, ..., 1, 0, 0])

# Creating Train and Test Splits

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, random_state = 52594, stratify = y)

# Finding the number of made and missed field goals in the training data, as well as the percentage of field goals made

In [12]:
import numpy as np

In [13]:
np.unique(ytrain, return_counts = True)

(array([0, 1]), array([3297,  607], dtype=int64))

In [14]:
3297/(3297 + 607)

0.8445184426229508

# Encoding the training data with OneHotEncoding

In [15]:
CatVar = ['posteam', 'game_half', 'kicker_player_name', 'stadium', 'weather_detail']
OtherVar = ['half_seconds_remaining', "kick_distance", "score_differential", "weather_temperature", "weather_wind_mph"]

In [16]:
cat = utils.get_cat(CatVar, X)#Gets the possible values for everything in the FullFrame so encoding doesn't result in different column numbers
enc_Xtrain = utils.ohe(CatVar, OtherVar, Xtrain, cat)
enc_Xtest = utils.ohe(CatVar, OtherVar, Xtest, cat)

# Scaling the data

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [18]:
scaler.fit(enc_Xtrain)
scaled_enc_Xtrain = scaler.transform(enc_Xtrain)
scaled_enc_Xtest = scaler.transform(enc_Xtest)

In [19]:
scaled_enc_Xtrain = pd.DataFrame(scaled_enc_Xtrain, columns = enc_Xtrain.columns)
scaled_enc_Xtrain

,posteam_SEA,posteam_GB,posteam_NO,posteam_ATL,posteam_MIN,posteam_STL,posteam_PIT,posteam_CLE,posteam_JAX,posteam_PHI,...,weather_detail_DOME (Open Roof),weather_detail_Rain,weather_detail_Fog,weather_detail_Snow,weather_detail_Rain | Fog,half_seconds_remaining,kick_distance,score_differential,weather_temperature,weather_wind_mph
0,-0.185597,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,5.475776,-0.167869,-0.184115,-0.180364,...,-0.144639,-0.218569,-0.076983,-0.050676,-0.027731,-1.320053,0.689907,0.252831,0.732052,-0.437333
1,-0.185597,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,-0.182623,-0.167869,-0.184115,-0.180364,...,-0.144639,-0.218569,-0.076983,-0.050676,-0.027731,1.716311,-1.471177,1.672105,-1.294714,-0.664344
2,-0.185597,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,-0.182623,-0.167869,-0.184115,-0.180364,...,-0.144639,-0.218569,-0.076983,-0.050676,-0.027731,1.009736,-0.881790,0.689530,0.922061,0.016689
3,5.388021,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,-0.182623,-0.167869,-0.184115,-0.180364,...,-0.144639,-0.218569,-0.076983,-0.050676,-0.027731,0.881788,-0.488866,0.362006,-0.091322,-1.118366
4,-0.185597,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,-0.182623,5.957011,-0.184115,-0.180364,...,-0.144639,-0.218569,-0.076983,-0.050676,-0.027731,0.956265,-1.569408,-0.074694,-0.344668,1.378754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,-0.185597,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,-0.182623,-0.167869,-0.184115,-0.180364,...,-0.144639,-0.218569,-0.076983,-0.050676,-0.027731,-1.213112,-0.194172,-1.166444,-1.738069,-0.437333
3900,-0.185597,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,-0.182623,-0.167869,-0.184115,-0.180364,...,-0.144639,-0.218569,-0.076983,-0.050676,-0.027731,-1.278040,1.377525,-1.275619,1.745435,0.470710
3901,-0.185597,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,-0.182623,-0.167869,-0.184115,-0.180364,...,-0.144639,-0.218569,-0.076983,-0.050676,-0.027731,-0.118875,-0.095941,0.689530,0.605379,-1.118366
3902,-0.185597,-0.177313,-0.174994,-0.180364,-0.191425,-0.11505,-0.182623,-0.167869,-0.184115,-0.180364,...,6.913754,-0.218569,-0.076983,-0.050676,-0.027731,1.141502,0.984601,-0.074694,0.858725,-0.664344


# Building a base logistic regression model

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
BaseLogReg = utils.cross_validate(scaled_enc_Xtrain, ytrain, "LogReg", penalty = 'none', score = 'f1')

In [22]:
#The f1 score per cross validation test. The scores are low and highly variable

BaseLogReg

[0.11363636363636363,
 0.24691358024691357,
 0.38095238095238093,
 0.27906976744186046,
 0.18421052631578946,
 0.17283950617283952,
 0.21176470588235294,
 0.25316455696202533,
 0.3333333333333333,
 0.2278481012658228]

In [23]:
utils.get_average_score(BaseLogReg)

0.24037328222096818

In [24]:
#Repeating for classification accuracy. It looks decent, but it is lower than predicting win on every kick

BaseLogReg = utils.cross_validate(scaled_enc_Xtrain, ytrain, "LogReg", penalty = 'none')

In [25]:
BaseLogReg

[0.8005115089514067,
 0.8439897698209718,
 0.8670076726342711,
 0.8414322250639387,
 0.841025641025641,
 0.8282051282051283,
 0.8282051282051283,
 0.8487179487179487,
 0.8358974358974359,
 0.8435897435897436]

In [26]:
utils.get_average_score(BaseLogReg)

0.8378582202111613

In [27]:
#Confusion matrix for this model on the test data
BaseLogRegFit = LogisticRegression(penalty = 'none', max_iter = 10000)
BaseLogRegFit.fit(scaled_enc_Xtrain, ytrain)

ypred = BaseLogRegFit.predict(scaled_enc_Xtrain)

from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(ytrain, ypred)

matrix

array([[3217,   80],
       [ 467,  140]], dtype=int64)

In [28]:
from sklearn.metrics import f1_score

f1_score(ytrain, ypred)

0.33857315598548976

In [29]:
BaseLogRegFit.score(scaled_enc_Xtrain, ytrain)

0.8598872950819673

# Grid search for LogReg

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

In [32]:
LogRegGrid = LogisticRegression(solver = 'saga', max_iter=10000)

In [33]:
GridSearch = GridSearchCV(estimator = LogRegGrid, param_grid = grid, cv = 10, return_train_score = True, verbose = 2, scoring = 'f1')

In [34]:
GridSearch.fit(scaled_enc_Xtrain, ytrain)

Fitting 10 folds for each of 14 candidates, totalling 140 fits
[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] .........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l1 ............................................
[CV] ............................. C=0.0001, penalty=l1, total=   0.0s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ............................. C=0.0001, penalty=l2, total=   0.2s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ............................. C=0.0001, penalty=l2, total=   0.2s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ............................. C=0.0001, penalty=l2, total=   0.2s
[CV] C=0.0001, penalty=l2 ............................................
[CV] ............................. C=0.0001, penalty=l2, total=   0.2s
[CV] 

[CV] ................................ C=0.1, penalty=l1, total=   0.9s
[CV] C=0.1, penalty=l1 ...............................................
[CV] ................................ C=0.1, penalty=l1, total=   0.5s
[CV] C=0.1, penalty=l1 ...............................................
[CV] ................................ C=0.1, penalty=l1, total=   0.8s
[CV] C=0.1, penalty=l1 ...............................................
[CV] ................................ C=0.1, penalty=l1, total=   0.8s
[CV] C=0.1, penalty=l1 ...............................................
[CV] ................................ C=0.1, penalty=l1, total=   1.1s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ................................ C=0.1, penalty=l2, total=   1.7s
[CV] C=0.1, penalty=l2 ...............................................
[CV] ................................ C=0.1, penalty=l2, total=   1.6s
[CV] C=0.1, penalty=l2 ...............................................
[CV] .

[CV] ................................ C=100, penalty=l1, total=  10.7s
[CV] C=100, penalty=l1 ...............................................
[CV] ................................ C=100, penalty=l1, total=  10.4s
[CV] C=100, penalty=l1 ...............................................
[CV] ................................ C=100, penalty=l1, total=  10.9s
[CV] C=100, penalty=l1 ...............................................
[CV] ................................ C=100, penalty=l1, total=   9.2s
[CV] C=100, penalty=l1 ...............................................
[CV] ................................ C=100, penalty=l1, total=   9.7s
[CV] C=100, penalty=l1 ...............................................
[CV] ................................ C=100, penalty=l1, total=  11.0s
[CV] C=100, penalty=l1 ...............................................
[CV] ................................ C=100, penalty=l1, total=  10.1s
[CV] C=100, penalty=l2 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed:  7.3min finished


GridSearchCV(cv=10, estimator=LogisticRegression(max_iter=10000, solver='saga'),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2']},
             return_train_score=True, scoring='f1', verbose=2)

In [35]:
pd.DataFrame(GridSearch.cv_results_).sort_values('mean_test_score', ascending = False).T

,11,12,13,9,10,8,7,6,5,4,0,1,2,3
mean_fit_time,6.06196,10.4089,6.61894,3.90017,8.3824,5.03598,1.51929,0.816783,0.338818,0.392614,0.0174539,0.197472,0.0487712,0.214678
std_fit_time,0.463779,0.720944,0.392772,0.483544,0.900981,0.939715,0.244992,0.153477,0.0436697,0.0687993,0.00581925,0.0281455,0.0110576,0.0380853
mean_score_time,0.00307672,0.00312378,0.00288675,0.0029768,0.00296919,0.00298643,0.00298061,0.00298693,0.00307977,0.00297921,0.00318856,0.00319166,0.00299201,0.00309174
std_score_time,0.000296638,0.000581251,0.000298681,1.29157e-05,1.17712e-05,0.000439551,1.33191e-05,0.000439878,0.000303477,1.29389e-05,0.000594441,0.000398898,0.000630827,0.00053718
param_C,10,100,100,1,10,1,0.1,0.1,0.01,0.01,0.0001,0.0001,0.001,0.001
param_penalty,l2,l1,l2,l2,l1,l1,l2,l1,l2,l1,l1,l2,l1,l2
params,"{'C': 10, 'penalty': 'l2'}","{'C': 100, 'penalty': 'l1'}","{'C': 100, 'penalty': 'l2'}","{'C': 1, 'penalty': 'l2'}","{'C': 10, 'penalty': 'l1'}","{'C': 1, 'penalty': 'l1'}","{'C': 0.1, 'penalty': 'l2'}","{'C': 0.1, 'penalty': 'l1'}","{'C': 0.01, 'penalty': 'l2'}","{'C': 0.01, 'penalty': 'l1'}","{'C': 0.0001, 'penalty': 'l1'}","{'C': 0.0001, 'penalty': 'l2'}","{'C': 0.001, 'penalty': 'l1'}","{'C': 0.001, 'penalty': 'l2'}"
split0_test_score,0.113636,0.113636,0.113636,0.113636,0.113636,0.117647,0.119048,0.103896,0.084507,0,0,0,0,0
split1_test_score,0.246914,0.246914,0.246914,0.246914,0.246914,0.246914,0.253165,0.216216,0.115942,0.0322581,0,0,0,0
split2_test_score,0.380952,0.380952,0.380952,0.361446,0.361446,0.365854,0.329114,0.263158,0.222222,0.0322581,0,0,0,0


In [36]:
#Confusion matrix for the winning model from the grid search
GridLogRegFit = LogisticRegression(C = 10, penalty = 'l2', max_iter = 10000)
GridLogRegFit.fit(scaled_enc_Xtrain, ytrain)

ypred = GridLogRegFit.predict(scaled_enc_Xtrain)

from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(ytrain, ypred)

matrix

array([[3217,   80],
       [ 468,  139]], dtype=int64)

In [37]:
#This model is almost identical to the base one, performing slightly worse on the confusion matrix

In [38]:
f1_score(ytrain, ypred)

0.3365617433414044

In [39]:
GridLogRegFit.score(scaled_enc_Xtrain, ytrain)

0.8596311475409836

# Installing imbalanced learn for oversampling

In [40]:
pip install -U imbalanced-learn

Requirement already up-to-date: imbalanced-learn in c:\users\michael\anaconda3\lib\site-packages (0.7.0)
Note: you may need to restart the kernel to use updated packages.


# Importing random over sampler

In [41]:
from imblearn.over_sampling import RandomOverSampler

# Using RandomOverSampler to over sample the data to improve the f1 score

In [42]:
#Using f1 as the scorer. The values used for this were determined by experimentation, looking for the best combination of f1 and classification accuracy across the cross validations

ROSLogReg = utils.cross_validate(scaled_enc_Xtrain, ytrain, "LogReg", penalty = 'l1', solver = 'saga', oversample = 'ros', ss = .6, score = 'f1')

ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4748, 158)
ROS
(4748, 158)
ROS
(4748, 158)


In [43]:
utils.get_average_score(ROSLogReg)

0.425083906150979

In [44]:
ROSLogReg

[0.4,
 0.4177215189873418,
 0.4966442953020135,
 0.43636363636363634,
 0.41891891891891897,
 0.3647058823529411,
 0.39473684210526316,
 0.45833333333333337,
 0.46341463414634143,
 0.4]

In [45]:
#Redoing the above to get the classification accuracy score

ROSLogReg = utils.cross_validate(scaled_enc_Xtrain, ytrain, "LogReg", penalty = 'l1', solver = 'saga', oversample = 'ros', CatVar = CatVar, ss = .6, score = 'acc')

ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4748, 158)
ROS
(4748, 158)
ROS
(4748, 158)


In [46]:
utils.get_average_score(ROSLogReg)

0.7684405534789167

In [47]:
ROSLogReg

[0.7468030690537084,
 0.7647058823529411,
 0.8081841432225064,
 0.7621483375959079,
 0.7794871794871795,
 0.7230769230769231,
 0.764102564102564,
 0.8,
 0.7743589743589744,
 0.7615384615384615]

In [48]:
#Applying the ROS to the testing data

ros = RandomOverSampler(random_state = 52594, sampling_strategy = .6)
rosX, rosy = ros.fit_resample(scaled_enc_Xtrain, ytrain)
rosLogReg = LogisticRegression(penalty = 'l1', solver = 'saga', max_iter = 10000).fit(rosX, rosy)

In [49]:
ypred = rosLogReg.predict(scaled_enc_Xtrain)

f1_score(ytrain, ypred)

0.5038071065989848

In [50]:
rosLogReg.score(scaled_enc_Xtrain, ytrain)

0.7996926229508197

In [51]:
matrix = confusion_matrix(ytrain, ypred)

matrix

array([[2725,  572],
       [ 210,  397]], dtype=int64)

# Making a decision tree model using grid search

In [52]:
from sklearn.tree import DecisionTreeClassifier

In [53]:
DecisionTree = DecisionTreeClassifier(max_leaf_nodes = 100, min_samples_leaf= 200)
DecisionTree.fit(enc_Xtrain, ytrain)

DecisionTreeClassifier(max_leaf_nodes=100, min_samples_leaf=200)

In [54]:
from sklearn.model_selection import cross_validate

In [55]:
dtcv = cross_validate(DecisionTree, scaled_enc_Xtrain, ytrain, cv= 10, return_train_score = True, return_estimator = True, verbose = 2, scoring = 'f1')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished


In [56]:
#This suggests the model is simply predicting "make" for every kick, which will not work for the purposes of this project

dtcv['test_score']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# Using grid search for Decision Tree

In [57]:
dtgrid = [{'max_depth':[1,2,3,4,5,6,7,8,9,10,15,20,25,30], 'max_features' :[1,2,3,4,5,6,7,8,9,10,15,20,25,30]}]

gridsearchdt = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = dtgrid, scoring = 'f1', cv = 10, verbose = 2)

griddt = gridsearchdt.fit(enc_Xtrain, ytrain)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 10 folds for each of 196 candidates, totalling 1960 fits
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.0s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.0s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.0s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.0s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.0s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.0s
[CV] max_depth=1, max_features=1 .....................................
[CV] .......

[CV] ...................... max_depth=1, max_features=7, total=   0.0s
[CV] max_depth=1, max_features=8 .....................................
[CV] ...................... max_depth=1, max_features=8, total=   0.0s
[CV] max_depth=1, max_features=8 .....................................
[CV] ...................... max_depth=1, max_features=8, total=   0.0s
[CV] max_depth=1, max_features=8 .....................................
[CV] ...................... max_depth=1, max_features=8, total=   0.0s
[CV] max_depth=1, max_features=8 .....................................
[CV] ...................... max_depth=1, max_features=8, total=   0.0s
[CV] max_depth=1, max_features=8 .....................................
[CV] ...................... max_depth=1, max_features=8, total=   0.0s
[CV] max_depth=1, max_features=8 .....................................
[CV] ...................... max_depth=1, max_features=8, total=   0.0s
[CV] max_depth=1, max_features=8 .....................................
[CV] .

[CV] ..................... max_depth=1, max_features=30, total=   0.0s
[CV] max_depth=1, max_features=30 ....................................
[CV] ..................... max_depth=1, max_features=30, total=   0.0s
[CV] max_depth=1, max_features=30 ....................................
[CV] ..................... max_depth=1, max_features=30, total=   0.0s
[CV] max_depth=1, max_features=30 ....................................
[CV] ..................... max_depth=1, max_features=30, total=   0.0s
[CV] max_depth=1, max_features=30 ....................................
[CV] ..................... max_depth=1, max_features=30, total=   0.0s
[CV] max_depth=2, max_features=1 .....................................
[CV] ...................... max_depth=2, max_features=1, total=   0.0s
[CV] max_depth=2, max_features=1 .....................................
[CV] ...................... max_depth=2, max_features=1, total=   0.0s
[CV] max_depth=2, max_features=1 .....................................
[CV] .

[CV] ...................... max_depth=2, max_features=7, total=   0.0s
[CV] max_depth=2, max_features=7 .....................................
[CV] ...................... max_depth=2, max_features=7, total=   0.0s
[CV] max_depth=2, max_features=7 .....................................
[CV] ...................... max_depth=2, max_features=7, total=   0.0s
[CV] max_depth=2, max_features=7 .....................................
[CV] ...................... max_depth=2, max_features=7, total=   0.0s
[CV] max_depth=2, max_features=7 .....................................
[CV] ...................... max_depth=2, max_features=7, total=   0.0s
[CV] max_depth=2, max_features=8 .....................................
[CV] ...................... max_depth=2, max_features=8, total=   0.0s
[CV] max_depth=2, max_features=8 .....................................
[CV] ...................... max_depth=2, max_features=8, total=   0.0s
[CV] max_depth=2, max_features=8 .....................................
[CV] .

...

[CV] ..................... max_depth=25, max_features=5, total=   0.0s
[CV] max_depth=25, max_features=5 ....................................
[CV] ..................... max_depth=25, max_features=5, total=   0.0s
[CV] max_depth=25, max_features=5 ....................................
[CV] ..................... max_depth=25, max_features=5, total=   0.0s
[CV] max_depth=25, max_features=6 ....................................
[CV] ..................... max_depth=25, max_features=6, total=   0.0s
[CV] max_depth=25, max_features=6 ....................................
[CV] ..................... max_depth=25, max_features=6, total=   0.0s
[CV] max_depth=25, max_features=6 ....................................
[CV] ..................... max_depth=25, max_features=6, total=   0.0s
[CV] max_depth=25, max_features=6 ....................................
[CV] ..................... max_depth=25, max_features=6, total=   0.0s
[CV] max_depth=25, max_features=6 ....................................
[CV] .

[CV] .................... max_depth=25, max_features=20, total=   0.0s
[CV] max_depth=25, max_features=20 ...................................
[CV] .................... max_depth=25, max_features=20, total=   0.0s
[CV] max_depth=25, max_features=20 ...................................
[CV] .................... max_depth=25, max_features=20, total=   0.0s
[CV] max_depth=25, max_features=20 ...................................
[CV] .................... max_depth=25, max_features=20, total=   0.0s
[CV] max_depth=25, max_features=20 ...................................
[CV] .................... max_depth=25, max_features=20, total=   0.0s
[CV] max_depth=25, max_features=20 ...................................
[CV] .................... max_depth=25, max_features=20, total=   0.0s
[CV] max_depth=25, max_features=20 ...................................
[CV] .................... max_depth=25, max_features=20, total=   0.0s
[CV] max_depth=25, max_features=20 ...................................
[CV] .

[CV] ..................... max_depth=30, max_features=4, total=   0.0s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.0s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.0s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.0s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.0s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.0s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.0s
[CV] max_depth=30, max_features=5 ....................................
[CV] .

[CV] .................... max_depth=30, max_features=15, total=   0.0s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.0s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.0s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.0s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.0s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.0s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.0s
[CV] max_depth=30, max_features=15 ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 1960 out of 1960 | elapsed:   26.3s finished


In [58]:
pd.DataFrame(gridsearchdt.cv_results_).sort_values('mean_test_score', ascending = False).T

,195,194,192,181,193,180,179,178,189,164,...,15,17,44,19,43,42,34,1,31,0
mean_fit_time,0.0218408,0.0194649,0.0156232,0.0209462,0.0179553,0.0191558,0.0164647,0.0143645,0.0124766,0.0138695,...,0.00731664,0.00749981,0.00748246,0.00741146,0.00748086,0.00718734,0.00767858,0.00738468,0.00778542,0.00759094
std_fit_time,0.00163858,0.00080343,0.000621038,0.0011018,0.000784625,0.000605453,0.000500644,0.000486762,0.000505831,0.000532791,...,0.000624812,0.000815649,0.000489359,0.000517462,0.000495131,0.000586731,0.000443844,0.000488486,0.00059088,0.000480125
mean_score_time,0.0030915,0.00308785,0.00269017,0.00299356,0.00288782,0.00309062,0.00289271,0.00299587,0.00268819,0.00279324,...,0.00285895,0.00279884,0.00259359,0.0026644,0.00269196,0.00269561,0.00259039,0.00228827,0.0025831,0.00259008
std_score_time,0.000532002,0.000309631,0.000454307,1.84224e-05,0.000297979,0.000696746,0.00052687,1.21379e-05,0.000460931,0.000388223,...,0.000312482,0.000399677,0.000488725,0.00044315,0.000457103,0.000446137,0.0004799,0.000440465,0.000489504,0.000486224
param_max_depth,30,30,30,25,30,25,25,25,30,20,...,2,2,4,2,4,4,3,1,3,1
param_max_features,30,25,15,30,20,25,20,15,8,15,...,2,4,3,6,2,1,7,2,4,1
params,"{'max_depth': 30, 'max_features': 30}","{'max_depth': 30, 'max_features': 25}","{'max_depth': 30, 'max_features': 15}","{'max_depth': 25, 'max_features': 30}","{'max_depth': 30, 'max_features': 20}","{'max_depth': 25, 'max_features': 25}","{'max_depth': 25, 'max_features': 20}","{'max_depth': 25, 'max_features': 15}","{'max_depth': 30, 'max_features': 8}","{'max_depth': 20, 'max_features': 15}",...,"{'max_depth': 2, 'max_features': 2}","{'max_depth': 2, 'max_features': 4}","{'max_depth': 4, 'max_features': 3}","{'max_depth': 2, 'max_features': 6}","{'max_depth': 4, 'max_features': 2}","{'max_depth': 4, 'max_features': 1}","{'max_depth': 3, 'max_features': 7}","{'max_depth': 1, 'max_features': 2}","{'max_depth': 3, 'max_features': 4}","{'max_depth': 1, 'max_features': 1}"
split0_test_score,0.25,0.27027,0.273504,0.348624,0.280374,0.18,0.326923,0.236559,0.259259,0.216867,...,0,0,0,0,0,0,0,0,0,0
split1_test_score,0.205128,0.188034,0.324786,0.222222,0.237624,0.238532,0.188679,0.229885,0.263736,0.321839,...,0,0,0,0,0,0,0,0,0,0
split2_test_score,0.346457,0.325203,0.309091,0.315789,0.310345,0.241758,0.242991,0.291667,0.206186,0.244898,...,0,0,0,0,0,0,0,0,0,0


# Using Random Over Sample on Decision Tree Model

In [78]:
#Using f1 as the scorer. The values used for this were determined by experimentation, looking for the best combination of f1 and classification accuracy across the cross validations

ROSDecisionTree = utils.cross_validate(scaled_enc_Xtrain, ytrain, "Tree", penalty = 'l1', solver = 'saga', oversample = 'ros', ss = .6, score = 'f1')

ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4748, 158)
ROS
(4748, 158)
ROS
(4748, 158)


In [79]:
ROSDecisionTree

[0.4157303370786517,
 0.42168674698795183,
 0.4654088050314465,
 0.37931034482758624,
 0.3582089552238806,
 0.435374149659864,
 0.41379310344827586,
 0.43209876543209874,
 0.3614457831325301,
 0.38028169014084506]

In [80]:
utils.get_average_score(ROSDecisionTree)

0.4063338680963131

In [75]:
#Using f1 as the scorer. The values used for this were determined by experimentation, looking for the best combination of f1 and classification accuracy across the cross validations

ROSDecisionTree = utils.cross_validate(scaled_enc_Xtrain, ytrain, "Tree", penalty = 'l1', solver = 'saga', oversample = 'ros', ss = .6, score = 'acc')

ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4747, 158)
ROS
(4748, 158)
ROS
(4748, 158)
ROS
(4748, 158)


In [76]:
ROSDecisionTree

[0.7340153452685422,
 0.7544757033248082,
 0.782608695652174,
 0.7237851662404092,
 0.7794871794871795,
 0.7871794871794872,
 0.782051282051282,
 0.764102564102564,
 0.7282051282051282,
 0.7743589743589744]

In [77]:
utils.get_average_score(ROSDecisionTree)

0.7610269525870549

# Building Random Forest Model

In [62]:
from sklearn.ensemble import RandomForestClassifier

In [63]:
RanFor = RandomForestClassifier(n_estimators = 700, max_features = 'auto', max_depth = 20)

In [64]:
rfcv = cross_validate(RanFor, scaled_enc_Xtrain, ytrain, cv= 10, return_train_score = True, return_estimator = True, verbose = 2, scoring = "f1")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=   3.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.7s remaining:    0.0s


[CV] ................................................. , total=   3.5s
[CV]  ................................................................
[CV] ................................................. , total=   3.5s
[CV]  ................................................................
[CV] ................................................. , total=   3.5s
[CV]  ................................................................
[CV] ................................................. , total=   3.5s
[CV]  ................................................................
[CV] ................................................. , total=   3.5s
[CV]  ................................................................
[CV] ................................................. , total=   3.5s
[CV]  ................................................................
[CV] ................................................. , total=   3.5s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   38.1s finished


In [65]:
#These scores are very low

rfcv['test_score']

array([0.06153846, 0.03125   , 0.09230769, 0.09230769, 0.        ,
       0.        , 0.03174603, 0.03076923, 0.12121212, 0.09375   ])

# Using a grid search for a random forest model

In [66]:
rfcgrid = [{'max_depth':[1,2,3,4,5,6,7,8,9,10,15,20,25,30], 'max_features' :[1,2,3,4,5,6,7,8,9,10,15,20,25,30]}]

gridsearchrfc = GridSearchCV(estimator = RandomForestClassifier(), param_grid = rfcgrid, scoring = 'f1', cv = 10, verbose = 2)

gridRFC = gridsearchrfc.fit(enc_Xtrain, ytrain)

Fitting 10 folds for each of 196 candidates, totalling 1960 fits
[CV] max_depth=1, max_features=1 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ...................... max_depth=1, max_features=1, total=   0.1s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.1s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.1s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.2s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.1s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.1s
[CV] max_depth=1, max_features=1 .....................................
[CV] ...................... max_depth=1, max_features=1, total=   0.1s
[CV] max_depth=1, max_features=1 .....................................
[CV] .

[CV] ...................... max_depth=1, max_features=6, total=   0.2s
[CV] max_depth=1, max_features=6 .....................................
[CV] ...................... max_depth=1, max_features=6, total=   0.2s
[CV] max_depth=1, max_features=7 .....................................
[CV] ...................... max_depth=1, max_features=7, total=   0.2s
[CV] max_depth=1, max_features=7 .....................................
[CV] ...................... max_depth=1, max_features=7, total=   0.2s
[CV] max_depth=1, max_features=7 .....................................
[CV] ...................... max_depth=1, max_features=7, total=   0.2s
[CV] max_depth=1, max_features=7 .....................................
[CV] ...................... max_depth=1, max_features=7, total=   0.2s
[CV] max_depth=1, max_features=7 .....................................
[CV] ...................... max_depth=1, max_features=7, total=   0.2s
[CV] max_depth=1, max_features=7 .....................................
[CV] .

[CV] ..................... max_depth=1, max_features=20, total=   0.2s
[CV] max_depth=1, max_features=20 ....................................
[CV] ..................... max_depth=1, max_features=20, total=   0.2s
[CV] max_depth=1, max_features=20 ....................................
[CV] ..................... max_depth=1, max_features=20, total=   0.2s
[CV] max_depth=1, max_features=25 ....................................
[CV] ..................... max_depth=1, max_features=25, total=   0.2s
[CV] max_depth=1, max_features=25 ....................................
[CV] ..................... max_depth=1, max_features=25, total=   0.2s
[CV] max_depth=1, max_features=25 ....................................
[CV] ..................... max_depth=1, max_features=25, total=   0.2s
[CV] max_depth=1, max_features=25 ....................................
[CV] ..................... max_depth=1, max_features=25, total=   0.2s
[CV] max_depth=1, max_features=25 ....................................
[CV] .

...

[CV] ..................... max_depth=25, max_features=7, total=   0.4s
[CV] max_depth=25, max_features=7 ....................................
[CV] ..................... max_depth=25, max_features=7, total=   0.4s
[CV] max_depth=25, max_features=7 ....................................
[CV] ..................... max_depth=25, max_features=7, total=   0.4s
[CV] max_depth=25, max_features=7 ....................................
[CV] ..................... max_depth=25, max_features=7, total=   0.4s
[CV] max_depth=25, max_features=7 ....................................
[CV] ..................... max_depth=25, max_features=7, total=   0.4s
[CV] max_depth=25, max_features=8 ....................................
[CV] ..................... max_depth=25, max_features=8, total=   0.5s
[CV] max_depth=25, max_features=8 ....................................
[CV] ..................... max_depth=25, max_features=8, total=   0.5s
[CV] max_depth=25, max_features=8 ....................................
[CV] .

[CV] .................... max_depth=25, max_features=25, total=   0.9s
[CV] max_depth=25, max_features=25 ...................................
[CV] .................... max_depth=25, max_features=25, total=   0.9s
[CV] max_depth=25, max_features=25 ...................................
[CV] .................... max_depth=25, max_features=25, total=   0.9s
[CV] max_depth=25, max_features=25 ...................................
[CV] .................... max_depth=25, max_features=25, total=   0.9s
[CV] max_depth=25, max_features=25 ...................................
[CV] .................... max_depth=25, max_features=25, total=   0.9s
[CV] max_depth=25, max_features=25 ...................................
[CV] .................... max_depth=25, max_features=25, total=   0.9s
[CV] max_depth=25, max_features=25 ...................................
[CV] .................... max_depth=25, max_features=25, total=   0.9s
[CV] max_depth=25, max_features=30 ...................................
[CV] .

[CV] ..................... max_depth=30, max_features=5, total=   0.4s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.4s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.4s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.4s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.4s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.4s
[CV] max_depth=30, max_features=5 ....................................
[CV] ..................... max_depth=30, max_features=5, total=   0.4s
[CV] max_depth=30, max_features=5 ....................................
[CV] .

[CV] .................... max_depth=30, max_features=10, total=   0.5s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.7s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.7s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.7s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.7s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.7s
[CV] max_depth=30, max_features=15 ...................................
[CV] .................... max_depth=30, max_features=15, total=   0.7s
[CV] max_depth=30, max_features=15 ...................................
[CV] .

[Parallel(n_jobs=1)]: Done 1960 out of 1960 | elapsed: 10.1min finished


In [67]:
pd.DataFrame(gridsearchrfc.cv_results_).sort_values('mean_test_score', ascending = False).T

,195,181,194,193,180,179,167,166,192,153,...,61,62,63,64,65,66,67,68,69,98
mean_fit_time,1.00211,0.963053,0.890296,0.777373,0.851616,0.734676,0.882862,0.774954,0.655724,0.748143,...,0.194242,0.197585,0.20086,0.210351,0.222576,0.254071,0.288898,0.327853,0.361699,0.166346
std_fit_time,0.0078074,0.0111417,0.00875602,0.0118065,0.00813749,0.00629129,0.00634704,0.00789153,0.00394937,0.00718999,...,0.00380448,0.00351649,0.00266909,0.00311701,0.00488147,0.00283592,0.00224529,0.00491551,0.00600063,0.00399586
mean_score_time,0.0164495,0.0165515,0.0162534,0.0163574,0.0159664,0.0163566,0.0155563,0.0154608,0.0172501,0.0148646,...,0.0127058,0.01257,0.0126667,0.0121694,0.0122684,0.0122643,0.0125755,0.0124292,0.0124169,0.012572
std_score_time,0.000496983,0.000658151,0.000452624,0.00048782,0.000435955,0.000795058,0.000651882,0.000502882,0.000771063,0.000944647,...,0.000586374,0.000651651,0.000458833,0.000868734,0.000642425,0.000459796,0.000797968,0.000578233,0.000477567,0.000665887
param_max_depth,30,25,30,30,25,25,20,20,30,15,...,5,5,5,5,5,5,5,5,5,8
param_max_features,30,30,25,20,25,20,30,25,15,30,...,6,7,8,9,10,15,20,25,30,1
params,"{'max_depth': 30, 'max_features': 30}","{'max_depth': 25, 'max_features': 30}","{'max_depth': 30, 'max_features': 25}","{'max_depth': 30, 'max_features': 20}","{'max_depth': 25, 'max_features': 25}","{'max_depth': 25, 'max_features': 20}","{'max_depth': 20, 'max_features': 30}","{'max_depth': 20, 'max_features': 25}","{'max_depth': 30, 'max_features': 15}","{'max_depth': 15, 'max_features': 30}",...,"{'max_depth': 5, 'max_features': 6}","{'max_depth': 5, 'max_features': 7}","{'max_depth': 5, 'max_features': 8}","{'max_depth': 5, 'max_features': 9}","{'max_depth': 5, 'max_features': 10}","{'max_depth': 5, 'max_features': 15}","{'max_depth': 5, 'max_features': 20}","{'max_depth': 5, 'max_features': 25}","{'max_depth': 5, 'max_features': 30}","{'max_depth': 8, 'max_features': 1}"
split0_test_score,0.219512,0.181818,0.179487,0.236842,0.202532,0.109589,0.184211,0.157895,0.166667,0.142857,...,0,0,0,0,0,0,0,0,0,0
split1_test_score,0.25974,0.114286,0.0833333,0.0857143,0.115942,0.115942,0.0588235,0.115942,0.0869565,0.0615385,...,0,0,0,0,0,0,0,0,0,0
split2_test_score,0.25,0.189189,0.24,0.169014,0.189189,0.2,0.197183,0.216216,0.171429,0.197183,...,0,0,0,0,0,0,0,0,0,0


# Using Random Over Sampler for Random Forest

In [106]:
#Using f1 as the scorer. The values used for this were determined by experimentation, looking for the best combination of f1 and classification accuracy across the cross validations

ROSRandomForest = utils.cross_validate(scaled_enc_Xtrain, ytrain, "RF", penalty = 'l1', solver = 'saga', oversample = 'ros', ss = 1.0, score = 'f1')

ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5936, 158)
ROS
(5936, 158)
ROS
(5936, 158)


In [107]:
ROSRandomForest

[0.380952380952381,
 0.43786982248520717,
 0.47904191616766467,
 0.39520958083832336,
 0.41830065359477125,
 0.3804347826086956,
 0.3875,
 0.36585365853658536,
 0.41916167664670656,
 0.3506493506493507]

In [108]:
utils.get_average_score(ROSRandomForest)

0.40149738224796855

In [109]:
#Again for accuracy

ROSRandomForest = utils.cross_validate(scaled_enc_Xtrain, ytrain, "RF", penalty = 'l1', solver = 'saga', oversample = 'ros', ss = 1.0, score = 'acc')

ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5934, 158)
ROS
(5936, 158)
ROS
(5936, 158)
ROS
(5936, 158)


In [110]:
ROSRandomForest

[0.7416879795396419,
 0.7647058823529411,
 0.7647058823529411,
 0.7237851662404092,
 0.7512820512820513,
 0.6974358974358974,
 0.7615384615384615,
 0.735897435897436,
 0.7487179487179487,
 0.7564102564102564]

In [111]:
utils.get_average_score(ROSRandomForest)

0.7446166961767985

# The best performing model is the over sampled linear regression model

# Fitting the model and using it on the test data

In [112]:
ros = RandomOverSampler(random_state = 52594, sampling_strategy = .6)
rosX, rosy = ros.fit_resample(scaled_enc_Xtrain, ytrain)
rosLogReg = LogisticRegression(penalty = 'l1', solver = 'saga', max_iter = 10000).fit(rosX, rosy)

In [113]:
ypred = rosLogReg.predict(scaled_enc_Xtest)

f1_score(ytest, ypred)

0.3631713554987212

In [114]:
rosLogReg.score(scaled_enc_Xtest, ytest)

0.7451381780962129

In [115]:
matrix = confusion_matrix(ytest, ypred)
matrix

array([[657, 168],
       [ 81,  71]], dtype=int64)

# Predicting 4 field goals from an NFL game (Ravens @ Eagles, 10/18/2020)

# Prediction 1, actual result: made

In [116]:
Predict1 = pd.DataFrame([['BAL', 732, 'Half1', 46.0, 14, "J.Tucker", "Lincoln Financial Field", 65, 7, "Normal"]], columns = X.columns)
enc_Predict1 = utils.ohe(CatVar, OtherVar, Predict1, cat)
scaled_enc_Predict1 = scaler.transform(enc_Predict1)
rosLogReg.predict_proba(scaled_enc_Predict1)

array([[0.76128494, 0.23871506]])

# Predicting the same field goal, but if it was the other team and other kicker

In [117]:
Predict2 = pd.DataFrame([['PHI', 732, 'Half1', 46.0, 14, "J.Elliott", "Lincoln Financial Field", 65, 7, "Normal"]], columns = X.columns)
enc_Predict2 = utils.ohe(CatVar, OtherVar, Predict2, cat)
scaled_enc_Predict2 = scaler.transform(enc_Predict2)
rosLogReg.predict_proba(scaled_enc_Predict2)

array([[0.44831699, 0.55168301]])

# While the percentages are suspect, it is clear that the model strongly accounts for the kicker and the kicking team

# Prediction 2, actual result: missed

In [118]:
Predict3 = pd.DataFrame([['PHI', 0, 'Half1', 52.0, -17, "J.Elliott", "Lincoln Financial Field", 65, 7, "Normal"]], columns = X.columns)
enc_Predict3 = utils.ohe(CatVar, OtherVar, Predict3, cat)
scaled_enc_Predict3 = scaler.transform(enc_Predict3)
rosLogReg.predict_proba(scaled_enc_Predict3)

array([[0.30425734, 0.69574266]])

# Predicting the same field goal, but other team other kicker

In [119]:
Predict4 = pd.DataFrame([['BAL', 0, 'Half1', 52.0, -17, "J.Tucker", "Lincoln Financial Field", 65, 7, "Normal"]], columns = X.columns)
enc_Predict4 = utils.ohe(CatVar, OtherVar, Predict4, cat)
scaled_enc_Predict4 = scaler.transform(enc_Predict4)
rosLogReg.predict_proba(scaled_enc_Predict4)

array([[0.63183665, 0.36816335]])

# Prediction 3, actual result: made

In [120]:
Predict5 = pd.DataFrame([['BAL', 601, 'Half2', 55.0, 10, "J.Tucker", "Lincoln Financial Field", 65, 7, "Normal"]], columns = X.columns)
enc_Predict5 = utils.ohe(CatVar, OtherVar, Predict5, cat)
scaled_enc_Predict5 = scaler.transform(enc_Predict5)
rosLogReg.predict_proba(scaled_enc_Predict5)

array([[0.50498134, 0.49501866]])

# Predicting the same field goal, but other team, other kicker

In [121]:
Predict6 = pd.DataFrame([['PHI', 601, 'Half2', 55.0, 10, "J.Elliott", "Lincoln Financial Field", 65, 7, "Normal"]], columns = X.columns)
enc_Predict6 = utils.ohe(CatVar, OtherVar, Predict6, cat)
scaled_enc_Predict6 = scaler.transform(enc_Predict6)
rosLogReg.predict_proba(scaled_enc_Predict6)

array([[0.20631474, 0.79368526]])

# Prediction 4, actual result: made

In [122]:
Predict7 = pd.DataFrame([['BAL', 437, 'Half2', 46.0, 13, "J.Tucker", "Lincoln Financial Field", 65, 7, "Normal"]], columns = X.columns)
enc_Predict7 = utils.ohe(CatVar, OtherVar, Predict7, cat)
scaled_enc_Predict7 = scaler.transform(enc_Predict7)
rosLogReg.predict_proba(scaled_enc_Predict7)

array([[0.76726073, 0.23273927]])

# Predicting the same field goal, but other team, other kicker

In [123]:
Predict8 = pd.DataFrame([['PHI', 437, 'Half2', 46.0, 13, "J.Elliott", "Lincoln Financial Field", 65, 7, "Normal"]], columns = X.columns)
enc_Predict8 = utils.ohe(CatVar, OtherVar, Predict8, cat)
scaled_enc_Predict8 = scaler.transform(enc_Predict8)
rosLogReg.predict_proba(scaled_enc_Predict8)

array([[0.45653442, 0.54346558]])